# **Regression**

El objetivo de esta aplicación de proyecto se puede separar en las siguientes fases:

1. **Definición del problema**. El estudiante debe definir un problema a ser resuelto, basado en la selección de un conjunto de datos de un problema que sea de su interés. Es importante que los datos seleccionados correspondan a un problema de regresión de forma que cumpla con el objetivo de la sección de este proyecto. Si la selección de los datos es complicada, como sugerencia pueden tomar el que sea de su interés de las siguientes páginas:

2. **Preparación de datos:** Limpieza y extracción de información estadística básica en los datos analizados. En esta fase las características de las bases de datos deben ser mencionadas (datos nulos, datos sesgados, datos atípicos, datos imbalanceados, distribución de datos, etc.), así como el proceso y decisiones tomadas para procesar y limpiar estos datos.  Incluir gráficos que muestren el comportamiento de los datos y explicar lo que se observa en el gráfico. 

3. **Procesar modelo de regresión:** Aplicar al menos 4 modelos de regresión para estimar el mapeo de los datos. La intención principal es hacer una comparación del performance de cada modelo, incluyendo la mención de la complejidad (número de parámetros y tiempo de ejecución). Añadir un método de ajuste de hiperparámetros para mejorar el performance del modelo. Crear un conjunto de datos "nuevos" donde se prediga la variable objetivo para esos nuevos datos. 

4. **Discusión de los resultados:** Basado en los resultados de la comparación y performance de los modelos, generar una sección de conclusiones donde se hable de las ventajas y desventajas de los modelos propuestos. 

## **Data** ##

In [2]:
import pandas as pd

In [19]:
columns = pd.read_csv('columns.csv').Feature.to_list()

with open('communities.data', 'r') as file:
    data = []
    for line in file:
        row = line.strip().split(',')
        data.append(row)


df = pd.DataFrame(data,columns=columns)


In [20]:
df.head(5)

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.9,0.12,...,0.12,0.26,0.2,0.06,0.04,0.9,0.5,0.32,0.14,0.2
1,53,?,?,Tukwilacity,1,0,0.16,0.12,0.74,0.45,...,0.02,0.12,0.45,?,?,?,?,0,?,0.67
2,24,?,?,Aberdeentown,1,0,0.42,0.49,0.56,0.17,...,0.01,0.21,0.02,?,?,?,?,0,?,0.43
3,34,5,81440,Willingborotownship,1,0.04,0.77,1,0.08,0.12,...,0.02,0.39,0.28,?,?,?,?,0,?,0.12
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,0.04,0.09,0.02,?,?,?,?,0,?,0.03
